In [14]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [15]:
# Load the TSV file, ignoring lines that start with '#'
articles = pd.read_csv('wikispeedia_paths-and-graph/articles.tsv', sep='\t', comment='#')
categories = pd.read_csv('wikispeedia_paths-and-graph/categories.tsv', sep='\t', comment='#')
links = pd.read_csv('wikispeedia_paths-and-graph/links.tsv', sep='\t', comment='#')
paths_finished = pd.read_csv('wikispeedia_paths-and-graph/paths_finished.tsv', sep='\t', comment='#')
paths_unfinished = pd.read_csv('wikispeedia_paths-and-graph/paths_unfinished.tsv', sep='\t', comment='#')
shortest_path = pd.read_csv('wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt', sep='\t', comment='#', header=None)

In [16]:
links.columns = ['linkSource', 'linkTarget']
paths_finished.columns = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating']
paths_unfinished.columns = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type']
paths_finished['status'] = 'finished'
paths_unfinished['status'] = 'unfinished'

# Concatenate the dataframes with the added 'status' column
concatenated_df = pd.concat(
    [
        paths_finished[['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating', 'status']],
        paths_unfinished[['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type', 'status']]
    ],
    ignore_index=True
)

In [17]:
# Create a copy of concatenated_df to avoid modifying the original DataFrame
clean_merge = concatenated_df.copy()

# Modify 'durationInSec' in the new DataFrame based on the condition
for index, row in clean_merge.iterrows():
    if row['type'] == 'timeout':  # Check if the last element in 'path' is 'timeout'
        clean_merge.loc[index, 'durationInSec'] -= 1800  # Subtract 1800 from 'durationInSec'

In [18]:
# Process each row based on 'status' and set 'target' accordingly
for i, row in clean_merge.iterrows():
    if row['status'] == 'finished':
        # Split the path and select the last element
        path_segments = row['path'].split(';') if pd.notnull(row['path']) else []
        row['target'] = path_segments[-1]

# To apply these changes to the DataFrame directly
clean_merge.loc[clean_merge['status'] == 'finished', 'target'] = clean_merge['path'].dropna().str.split(';').str[-1]


In [19]:
clean_merge.to_csv('clean_merge.csv')

In [20]:
clean_merge.sample(10)

,hashedIpAddress,timestamp,durationInSec,path,rating,status,target,type
2554,2438a4611e2f8038,1336925744,43,Brothers_Grimm;Germany;Belgium;Flanders,2.0,finished,Flanders,NaN
23352,0d57c8c57d75e2f5,1280350233,12,Apple;Fruit;Banana,NaN,finished,Banana,NaN
63187,4d4b3e8b7b19bb3d,1346879896,227,Asteroid;Mars;Mars_Reconnaissance_Orbiter;Mars...,NaN,unfinished,Viking,timeout
4038,0d57c8c57d75e2f5,1282504263,137,Edgar_Allan_Poe;Force;Galileo_Galilei;Astronom...,3.0,finished,Upsilon_Andromedae_b,NaN
49877,6d136e371e42474f,1232124163,85,Uruguay;South_America;Continent;Europe;Iceland...,2.0,finished,Viking,NaN
21545,3157a9d5499413c6,1381860036,176,Accountancy;Croatia;Democracy;United_States;Ba...,3.0,finished,Feather,NaN
56841,506c25f514f0bb19,1314005084,9,Rio_de_Janeiro,NaN,unfinished,Ozone,timeout
12306,473f51f614fad2ae,1302807704,83,Asteroid;Sun;Solar_System;Earth;United_States;...,3.0,finished,Viking,NaN
29401,65fa897479433d02,1251179850,189,Political_economy;Economics;Spain;Istanbul,2.0,finished,Istanbul,NaN
63611,5148774a1b2b666f,1348077146,111,White_cliffs_of_Dover;United_Kingdom;Great_Bri...,NaN,unfinished,James_II_of_England,timeout
